##IMPORT

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

tf.__version__
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 10814646953848160520, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 1975433551953419838
 physical_device_desc: "device: XLA_CPU device"]

##Load Data

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", reshape=False)
X_train, y_train           = mnist.train.images, mnist.train.labels
X_validation, y_validation = mnist.validation.images, mnist.validation.labels
X_test, y_test             = mnist.test.images, mnist.test.labels

assert(len(X_train) == len(y_train))
assert(len(X_validation) == len(y_validation))
assert(len(X_test) == len(y_test))

print()
print("Image Shape: {}".format(X_train[0].shape))
print()
print("Training Set:   {} samples".format(len(X_train)))
print("Validation Set: {} samples".format(len(X_validation)))
print("Test Set:       {} samples".format(len(X_test)))

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.

Image Shape: (28, 28, 1)

Training Set

##Preprocess the data

In [3]:
# Pad images with 0s
X_train      = np.pad(X_train, ((0,0),(2,2),(2,2),(0,0)), 'constant')
X_validation = np.pad(X_validation, ((0,0),(2,2),(2,2),(0,0)), 'constant')
X_test       = np.pad(X_test, ((0,0),(2,2),(2,2),(0,0)), 'constant')
    
print("Updated Image Shape: {}".format(X_train.shape))

Updated Image Shape: (55000, 32, 32, 1)


In [0]:
#Shuffle the training data
from sklearn.utils import shuffle

X_train, y_train = shuffle(X_train, y_train)

##Generator

In [0]:
def generator(Z, initializer):
    '''
    Takes an argument Z, an [M, 1, 1, 100] tensor of random numbers.
    Returns an operation that creates a generated image of shape [None, 32, 32, 1]
    '''

    with tf.variable_scope('generator'):

        # Layer 1 -> [None, 4, 4, 512]
        deconv_1 = tf.layers.conv2d_transpose(
            Z, 
            filters=512, 
            kernel_size=[4,4], 
            strides=[1,1], 
            padding='valid',
            kernel_initializer=initializer, 
            name='layer1')
        norm_1 = tf.layers.batch_normalization(deconv_1)
        lrelu_1 = tf.nn.leaky_relu(norm_1)

        # Layer 2 -> [None, 8, 8, 256]
        deconv_2 = tf.layers.conv2d_transpose(
            lrelu_1, 
            filters=256, 
            kernel_size=[4,4], 
            strides=[2,2], 
            padding='same', 
            kernel_initializer=initializer,
            name='layer2')
        norm_2 = tf.layers.batch_normalization(deconv_2)
        lrelu_2 = tf.nn.leaky_relu(norm_2)

        # Layer 3 -> [None, 16, 16, 128]
        deconv_3 = tf.layers.conv2d_transpose(
            lrelu_2,
            filters=128, 
            kernel_size=[4,4], 
            strides=[2,2], 
            padding='same', 
            kernel_initializer=initializer,
            name='layer3')
        norm_3 = tf.layers.batch_normalization(deconv_3)
        lrelu_3 = tf.nn.leaky_relu(norm_3)

        # Layer 4 -> [None, 32, 32, 1]
        deconv_4 = tf.layers.conv2d_transpose(
            lrelu_3, 
            filters=1,
            kernel_size=[4,4],
            strides=[2,2],
            padding='same',
            activation=tf.nn.tanh,
            kernel_initializer=initializer,
            name='layer4')
        output = tf.identity(deconv_4, name='generated_images')

        # Generated images of shape [M, 32, 32, 1]
        return output

### Discriminator

In [0]:
def discriminator(images, initializer, reuse=False):
    '''
    Takes an image as an argument [None, 32, 32, 1].
    Returns an operation that gives the probability of that image being 'real' [None, 1]
    '''

    with tf.variable_scope('discriminator', reuse=reuse):

        # Layer 1 -> [None, 16, 16, 128]
        conv_1 = tf.layers.conv2d(
            images, 
            filters=128, 
            kernel_size=[4,4], 
            strides=[2,2], 
            padding='same', 
            activation=tf.nn.leaky_relu,
            kernel_initializer=initializer, 
            name='layer1')

        # Layer 2 -> [None, 8, 8, 256]
        conv_2 = tf.layers.conv2d(
            conv_1, 
            filters=256, 
            kernel_size=[4,4], 
            strides=[2,2], 
            padding='same', 
            kernel_initializer=initializer,
            name='layer2')
        norm_2 = tf.layers.batch_normalization(conv_2)
        lrelu_2 = tf.nn.leaky_relu(norm_2)

        # Layer 3  -> [None, 4, 4, 512]
        conv_3 = tf.layers.conv2d(
            lrelu_2, 
            filters=512, 
            kernel_size=[4,4], 
            strides=[2,2], 
            padding='same', 
            kernel_initializer=initializer, 
            name='layer3')
        norm_3 = tf.layers.batch_normalization(conv_3)
        lrelu_3 = tf.nn.leaky_relu(norm_3)

        # Layer 5 -> [None, 1, 1, 1]
        conv_4 = tf.layers.conv2d(lrelu_3, filters=1, kernel_size=[4,4], strides=[1,1], padding='valid')
        sigmoid_4 = tf.nn.sigmoid(conv_4)
        output = tf.reshape(sigmoid_4, [-1, 1])

        return output

##Loss function

In [0]:
def loss(Dx, Dg):
    '''
    Dx = Probabilities assigned by D to the real images, [M, 1]
    Dg = Probabilities assigned by D to the generated images, [M, 1]
    '''
    with tf.variable_scope('loss'):
        loss_d = tf.identity(-tf.reduce_mean(tf.log(Dx) + tf.log(1. - Dg)), name='loss_d')
        loss_g = tf.identity(-tf.reduce_mean(tf.log(Dg)), name='loss_g')
        return loss_d, loss_g

##Train Operation

In [8]:
#set EPOCHS and BATCH for triaing
EPOCHS = 30     #30 entries
BATCH_SIZE = 256    #each entries have 128 samples
#set Learning rate for G and N
Rate_G = 0.0002
Rate_D = 0.0002


images_holder = tf.placeholder(tf.float32, shape=[None, 32, 32, 1], name='images_holder')
Z_holder = tf.placeholder(tf.float32, shape=[None, 1, 1, 100], name='z_holder')

# Forward
weights_initializer = tf.truncated_normal_initializer(stddev=0.02)
generated_images = generator(Z_holder, weights_initializer)
Dx = discriminator(images_holder, weights_initializer, False)
Dg = discriminator(generated_images, weights_initializer, True)
            
# Loss： Dx:real; Dg:Fake
loss_d, loss_g = loss(Dx, Dg)
            
# Optimize G or D
optimizer_g = tf.train.AdamOptimizer(learning_rate=Rate_G, beta1=0.5)
optimizer_d = tf.train.AdamOptimizer(learning_rate=Rate_D, beta1=0.5)
            
# Back propagation
g_vars = tf.trainable_variables(scope='generator')
d_vars = tf.trainable_variables(scope='discriminator')

# Training 
train_G = optimizer_g.minimize(loss_g, var_list=g_vars, name='train_G')
train_D = optimizer_d.minimize(loss_d, var_list = d_vars, name='train_D')


Instructions for updating:
Use keras.layers.conv2d_transpose instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.batch_normalization instead.
Instructions for updating:
Use keras.layers.conv2d instead.


##Main

In [0]:
with tf.Session() as sess:
    num_examples = len(X_train)
    sess.run(tf.global_variables_initializer())
    print("Training...")

    for i in range(EPOCHS):
        X_train = shuffle(X_train)
        
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            print("Get data")
            images = X_train[offset:end]
            images = images / 128.
            images = images - 1.
            Z = np.random.normal(0.0, 1.0, size=[images.shape[0], 1, 1, 100])
           # feed_dict = {'images_holder:0': batch_x 'z_holder:0': Z}
            print("Training G")
            sess.run(train_G,feed_dict={'images_holder:0': images, 'z_holder:0': Z})
            print("Training D")
            sess.run(train_D,feed_dict={'images_holder:0': images, 'z_holder:0': Z})
            
        if ((i+1)%10 == 0)or(i==0):
            print("EPOCH {} ...".format(i+1))
            #print("Validation Accuracy = {:.3f}".format(validation_accuracy))
            print()
   

print("END")
#plt.imshow((image[9, :, :, 0]+1)*128)
#plt.imshow(X_train[9].squeeze())
#https://github.com/sarahwolf32/DCGAN-for-MNIST/blob/master/trainer/task.py

Training...
Get data
Training G
Training D
Get data
Training G
Training D
Get data
Training G
